In [ ]:
import os 
dir_path = os.path.dirname(os.path.realpath("__file__"))
os.chdir(dir_path) 

In [ ]:
from data_collection import get_sym_data
from data_cleaning import foward_fill, process_returns
import database_manager as dm
from pykx import q, QError
os.chdir(dir_path) 


In [ ]:
# pull google stock data
sym = "googl"
sym = sym.upper()
table = get_sym_data(sym)
print(table)

In [ ]:
q.set('t',table)
q('count t')

In [ ]:
q('del:(count t)?2')
q('count del')

In [ ]:
# simuate missing values
table = q('update Close: ?[del=0; 0Nf; Close] from t')
print(table)

In [ ]:
# forward fill missing values
table = foward_fill(table)
print(table)

In [ ]:
# N_normalized price change %
# R_daily return %
# C_culminative returns %
table = process_returns(table)
q.set('t',table)
table = q('select Datetime, Open, N_Open, R_Open, C_Open from t')
table